In [ ]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_b3df59f4cfa4437ba5a0d8341462d910(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_e61abd9f360f5ae76429c3a3c8d7691ffe433702','domain': {'id': 'cc9350cb51284007801ed1b5d49f396e'},
            'password': 'ExnZeWW7-2,.,S#t'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df_data_1 = pd.read_csv(get_object_storage_file_with_credentials_b3df59f4cfa4437ba5a0d8341462d910('DefaultProjectsushidharjayaramanmavsutaedu', 'white.csv'))
df_data_1.head()


In [ ]:
if df_data_1["quality"] < 6:
    df_data_1["quality"] = "Low"
elif df_data_1["quality"] >= 7:
    df_data_1["quality"] = "Medium"
else df_data_1["quality"] = 6:
    df_data_1["quality"] = "High"
df_data_1.quality

In [ ]:
stratified_CV_data = df_data_1.sampleBy('quality', fractions={'Low': 1060./1640, 'High': 1.0, 'Medium' : 1060./2198}).cache()

stratified_CV_data.groupby('quality').count().toPandas()

In [ ]:
#final_CV_data = stratified_CV_data.drop('total sulfur dioxide').drop('density').cache()
final_CV_data = stratified_CV_data.drop('free sulfur dioxide').drop('density').cache()
#final_CV_data = stratified_CV_data.drop('total sulfur dioxide').drop('residual sugar').cache()
#final_CV_data = stratified_CV_data.drop('free sulfur dioxide').drop('residual sugar').cache()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
indexer = StringIndexer(inputCol="quality", outputCol="qualityIndex")
indexed = indexer.fit(final_CV_data).transform(final_CV_data)
indexed = indexed.drop('quality').cache()
indexed.show()

In [ ]:
stages = [] # stages in our Pipeline
  
label_stringIdx = StringIndexer(inputCol = "quality", outputCol = "label")
stages += [label_stringIdx]

#numericCols = ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","pH","sulphates","alcohol"]
numericCols = ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","total sulfur dioxide","free sulfur dioxide","density","pH","sulphates","alcohol"]
#numericCols = ["fixed acidity","volatile acidity","citric acid","chlorides","free sulfur dioxide","density","pH","sulphates","alcohol"]
#numericCols = ["fixed acidity","volatile acidity","citric acid","chlorides","total sulfur dioxide","density","pH","sulphates","alcohol"]
#assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
stages += [assembler]

cols = df_data_1.columns
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(df_data_1)
dataset = pipelineModel.transform(df_data_1)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
#display(dataset)
#type(dataset)
dataset.toPandas()